In [1]:
# Importation of liberies.
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import time
from scipy.interpolate import pchip_interpolate
from scipy.spatial.transform import Rotation as R
from scipy import interpolate, signal

In [4]:
# Functions
#_______________________________________________________________________________
def Load_Files(dir):
    data={}
    entries = os.listdir(dir)
    titles= ['Sensor','TimeStamp','Roll','Pitch','Yaw','xGyr','yGyr','zGyr',
             'BatteryLVL']
    for i in entries:
        if 'SENSOR'in i:
            name= i.split('.')
            s = pd.read_csv(dir + i, sep=",")
            s.columns= titles
            s['TimeStamp']= s['TimeStamp']/1000
            data[name[0]] = s
    return data
#_______________________________________________________________________________
def Plot_Data(data,test,phase):
  """
  This fuction plot the Roll, Pitch and Yaw data.
  """

  fig, ((ax1, axa), (ax2, axb), (ax3, axc)) = plt.subplots(3, 2, figsize=(9.6, 5.5))
  for i in data:
    s= data[i]
    ax1.plot(s['TimeStamp'],s['Roll'], label= i)
    ax2.plot(s['TimeStamp'],s['Pitch'], label= i)
    ax3.plot(s['TimeStamp'],s['Yaw'], label= i)
    axa.plot(s['TimeStamp'],s['xGyr'], label= i)
    axb.plot(s['TimeStamp'],s['yGyr'], label= i)
    axc.plot(s['TimeStamp'],s['zGyr'], label= i)
  fig.suptitle(phase+' '+test)
  ax1.set_xlabel('Time (s)')
  ax1.set_ylabel('Roll (°)')
  ax1.set_title('Roll')
  ax2.set_xlabel('Time (s)')
  ax2.set_ylabel('Pitch (°)')
  ax2.set_title('Pitch')
  ax3.set_xlabel('Time (s)')
  ax3.set_ylabel('Yaw (°)')
  ax3.set_title('Yaw')
  axa.set_xlabel('Time (s)')
  axa.set_ylabel('xGyr (°/s)')
  axa.set_title('xGyr')
  axb.set_xlabel('Time (s)')
  axb.set_ylabel('yGyr (°/s)')
  axb.set_title('yGyr')
  axc.set_xlabel('Time (s)')
  axc.set_ylabel('zGyr (°/s)')
  axc.set_title('zGyr')
  ax1.margins(0, 0.1)
  ax2.margins(0, 0.1)
  ax3.margins(0, 0.1)
  axa.margins(0, 0.1)
  axb.margins(0, 0.1)
  axc.margins(0, 0.1)
  fig.tight_layout()
  plt.show()
  return None
#_______________________________________________________________________________
def Set_Data(data):
  """
  This fuction set the sensors to start in 0.
  """
  for i in data:
    s= data[i]
    s= s.iloc[50:data[i].size-29,:]                                             #errase first 50 takes and last 29
    time= s["TimeStamp"]
    time= time-time.iloc[0]
    s['TimeStamp']= time
    data[i]= s
  return data 
#_______________________________________________________________________________
def Change_Range(data,thresh=89):
  """
  This fuction fix the angles lokking for the abrupt changes.
  """
  for s in data:
    n_d= data[s].loc[:,]
    for d in ['Roll','Pitch','Yaw']:
      var= n_d= data[s].loc[:,d]
      drv= np.where(((abs(var.diff()))>=thresh))
      drv=drv[0].tolist()
      if (len(drv) % 2) != 0: drv.append(len(n_d)-1)
      for f in range(0,len(drv),2):
        chg= var.iloc[drv[f]-1]+var.iloc[drv[f]]
        var.iloc[drv[f]:drv[(f)+1]]=-(var.iloc[drv[f]:drv[(f)+1]])+chg
      data[s].loc[:,d]= var
  return data
#_______________________________________________________________________________
def Resample(data, f):
  """
  This fuction resample the data to a frecuency passed by the user.
  """
  for i in data:
    s= data[i]
    n_t= s.loc[:,'TimeStamp']
    n_t= n_t-n_t.iloc[0]
    n_t= np.linspace(0,n_t.iloc[-1], round(n_t.iloc[-1]*f))
    r= s['Roll']
    p= s['Pitch']
    h= s['Yaw']
    x= s['xGyr']
    y= s['yGyr']
    z= s['zGyr']
    r= pchip_interpolate(s['TimeStamp'],r,n_t)
    p= pchip_interpolate(s['TimeStamp'],p,n_t)
    h= pchip_interpolate(s['TimeStamp'],h,n_t)
    x= pchip_interpolate(s['TimeStamp'],x,n_t)
    y= pchip_interpolate(s['TimeStamp'],y,n_t)
    z= pchip_interpolate(s['TimeStamp'],z,n_t)
    s= pd.DataFrame({'TimeStamp':n_t, 'Roll':r, 'Pitch':p, 'Yaw':h, 'xGyr':x, 
                     'yGyr':y, 'zGyr':z})
    data[i]= s
  return data
#_______________________________________________________________________________
def Syncronize(data):
  """
  This function synchronizes the sensors and returns the synchronized data 
  using cross correlation lag.
  """
  lag={}
  n= list(data.keys())
  l= len(data[n[0]])
  ref= n[0]
  for i in range(1,len(n)):
    if len(data[n[i]])<l:
      l= len(data[n[i]])
      ref= n[i]
  n.remove(ref)
  data_a= data[ref].loc[:,['xGyr','yGyr','zGyr']]
  lag=[0]
  for s in n:
    data_b= data[s].loc[:,['xGyr','yGyr','zGyr']]
    corr = signal.correlate(data_b, data_a, mode='same')
    lags = signal.correlation_lags(len(data_a), len(data_b),mode='same')
    corr_max= np.argmax(corr,axis=0)
    lag= np.append(lag,np.median([lags[corr_max[0]], lags[corr_max[1]], 
                                  lags[corr_max[2]]]))
  n= np.append(ref,n)
  if 0 > min(lag): lag += abs(min(lag))
  for s in range(0,len(n)):
    data[n[s]]= data[n[s]].iloc[int(lag[s]):,:]
    time= data[n[s]]['TimeStamp']
    time= time-time.iloc[0]
    data[n[s]]['TimeStamp']= time
  return data
#_______________________________________________________________________________
def Same_size(data):
  """
  This function makes all the dataframes have the same size.
  """
  n= list(data.keys())
  l= len(data[n[0]])
  for i in range(1,len(n)):
    if len(data[n[i]])<l:
      l= len(data[n[i]])
  for s in n:
    data[s]= data[s].iloc[0:l,:]
  return data

In [35]:
start_time = time.time()
#test= input('put the name test:')
test= 'SClau_IJose' #'RotCad_p1'
dir= "D:/_Proyecto-SAM/2022_10_18/Dia_1/"+test+'/' #"D:/_Proyecto-SAM/2022-02-01/"+test+'/'
# Load the data
data= Load_Files(dir)

# Plot the data
#Plot_Data(data,test,'Pre-Processing')
# Set Start to 0 Sensors(SAM).
data= Set_Data(data)
# Adjust angles range to a continous angles.
data= Change_Range(data,150)
# Oversample the data to 200 Hz.
data= Resample(data,200)
# Synchronize the data.
data= Syncronize(data)
# Resample the data to 50 Hz.
data= Resample(data,50)
# Make all the dataframes have the same size.
data= Same_size(data)
# Plot the data
#Plot_Data(data,test,'Post-Processing')

print("--- %s seconds ---" % (time.time() - start_time))

C:\Users\Jose Amorocho\AppData\Local\Temp\ipykernel_20452\992834514.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s['TimeStamp']= time
C:\Users\Jose Amorocho\AppData\Local\Temp\ipykernel_20452\992834514.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  var.iloc[drv[f]:drv[(f)+1]]=-(var.iloc[drv[f]:drv[(f)+1]])+chg


--- 0.62235426902771 seconds ---


In [45]:
#_______________________________________________________________________________
def Fle_Ext(x_vec):
  """
  This function calculates the flexion and extension angles using the vectors 
  of reference.
  """
  [xi,xs]= np.split(x_vec, 2)
  x_dot= np.dot(xi,xs)
  x_mag= np.linalg.norm(xi)*np.linalg.norm(xs) 
  return np.arccos(x_dot/x_mag)
#_______________________________________________________________________________
def Abd_Add(vec_ref):
  """ 
  This function calculates the abduction and adduction angles using the 
  reference plane whith a vector.
  """
  [a,b,r]= np.split(vec_ref, 3)
  n= np.cross(a, b)
  nr_dot= np.dot(n,r)
  nr_mag= np.linalg.norm(n)*np.linalg.norm(r) 
  return np.arcsin(nr_dot/nr_mag)
#_______________________________________________________________________________
def angles_Knee(ti,fe):
  """
  This function calculates the angles of the knee joint using the Tibia and 
  Femur sensors.
  """
  rti = R.from_euler('xyz',ti.loc[:,['Roll','Pitch','Yaw']],
        degrees=True).as_matrix()
  rfe = R.from_euler('xyz',fe.loc[:,['Roll','Pitch','Yaw']],
        degrees=True).as_matrix()
  x= np.concatenate((rti[:,0,:],-rfe[:,0,:]),axis=1)
  return pd.DataFrame(np.apply_along_axis(Fle_Ext,1,x),columns=['Fle_Ext'])
#_______________________________________________________________________________
def angles_Ankle(fo,ti,leg):
  """
  This function calculates the angles of the ankle joint using the Tibia and 
  Foot sensors.
  """
  rfo = R.from_euler('xyz',fo.loc[:,['Roll','Pitch','Yaw']],
        degrees=True).as_matrix()
  rti = R.from_euler('xyz',ti.loc[:,['Roll','Pitch','Yaw']],
        degrees=True).as_matrix()
  x= np.concatenate((rfo[:,0,:],-rti[:,0,:]),axis=1)
  if leg == 'left':xyx= np.concatenate((rfo[:,1,:],rfo[:,0,:],-rti[:,0,:]),
                                          axis=1)
  if leg == 'right':xyx= np.concatenate((rfo[:,0,:],-rfo[:,1,:],-rti[:,0,:]),
                                          axis=1)
  # Flexion-Extension angles.
  f_e= pd.DataFrame(np.apply_along_axis(Fle_Ext,1,x),columns=['Fle_Ext'])
  # Abduction-Adduction angles.
  a_a= pd.DataFrame(np.apply_along_axis(Abd_Add,1,xyx),columns=['Abd_Abb'])
  return pd.concat([f_e, a_a], axis=1)
#_______________________________________________________________________________
# Angle Knee.
ang_knee= angles_Knee(data['SENSOR06'],data['SENSOR07'])
# Angle Ankle.
ang_ankle= angles_Ankle(data['SENSOR08'],data['SENSOR06'],'right')

In [ ]:

rhi = R.from_euler('xyz',hi.loc[:,['Roll','Pitch','Yaw']],
      degrees=True).as_matrix()
rfe = R.from_euler('xyz',fe.loc[:,['Roll','Pitch','Yaw']],
      degrees=True).as_matrix()
x= np.concatenate((rhi[:,0,:],-rfe[:,0,:]),axis=1)

f_e = pd.DataFrame(np.apply_along_axis(Fle_Ext,1,x),columns=['Fle_Ext'])

In [ ]:
def angles_Hip(hi,fe):
  """
  This function calculates the angles of the knee joint using the Tibia and 
  Femur sensors.
  """
  rhi = R.from_euler('xyz',hi.loc[:,['Roll','Pitch','Yaw']],
        degrees=True).as_matrix()
  rfe = R.from_euler('xyz',fe.loc[:,['Roll','Pitch','Yaw']],
        degrees=True).as_matrix()
  x= np.concatenate((rhi[:,0,:],-rfe[:,0,:]),axis=1)
  pd.DataFrame(np.apply_along_axis(Fle_Ext,1,x),columns=['Fle_Ext'])